In [1]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Mega-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv("TweetsDataset.csv")
df=df.sample(5000)

In [3]:
def clean_and_preprocess(text):
    text = re.sub(r'@\w+', '', str(text))
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    stop_words = set(stopwords.words('italian'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    stemmer = SnowballStemmer('italian')
    words = [stemmer.stem(word) for word in words]
    words = [word for word in words if len(word) > 2]
    return " ".join(words)
df['clean_text'] = df['Content'].apply(clean_and_preprocess)


In [4]:
df['clean_text'] = df['Content'].apply(clean_and_preprocess)


In [5]:
df[['Content', 'clean_text']].head(10)


,Content,clean_text
130049,RT @Anittaop: @AnittaChart TERREMOTO NO SPOTIFY,terremot spotify
157855,"Sisma e ripristino delle scuole terremotate, e...",sism ripristin scuol terremot ecco lâordin diâ...
18598,RT @fatina909: ðŸ’” Vogliamo ricordarti cosÃ¬ ...,vogl ricord cosã kaos
188949,RT @MarioManca: Continuate a dedicare migliaia...,continu dedic migliai post allacqu evian firm ...
65911,"Una scossa di terremoto, di magnitudo compresa...",scoss terremot magnitud compres second prim st...
173585,RT @emergenzavvf: #Roma incendio ospedale San ...,incend ospedal san pietr aggiorn 1000 mancanz ...
189923,RT @UiltecSalerno: Un momento di analisi sulle...,moment analis problemat dellâacqu httpstco97im...
76586,Gente dellEmilia Romagna tutto apposto?,gent dellemil romagn appost
171157,RT @GallozziMrita: Dopo un violento incendio i...,dop violent incend can cas scompars gett dispe...
256992,RT @creTania_: Treno Italo bloccato 5 ore in g...,tren ital blocc ore galler parl odisse giorn n...


In [57]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")


# Fonction pour convertir le label en score binaire (0 ou 1)
def feel_it_score(text):
    try:
        result = classifier(text)[0]
        stars = int(result['label'][0])  # "5 stars" → 5
        return 1 if stars >= 3 else 0
    except Exception as e:
        print(f"Erreur sur: {text[:50]}... → {e}")
        return None

test_text = "Questo è un testo positivo."

try:
    result = classifier(test_text)[0]
    print(result)
except Exception as e:
    print(f"Erreur : {e}")
df['score'] = df['clean_text'].apply(feel_it_score)

# Aperçu
print(df[['Content', 'clean_text', 'score']].head())

Device set to use cpu


{'label': '5 stars', 'score': 0.5892539620399475}
                                                  Content  \
178197  RT @ImolaOggi: Varoufakis: lâ€™euro Ã¨ come un...   
260676  RT @CriminImmigratl: #CriminiImmigrati: Africa...   
65993   RT @Agenzia_Ansa: Scossa di #terremoto nel rea...   
252478  @renestrano Nessun tuono, nessun fulmine!  Sem...   
120612  RT @emergenzavvf: #Terremoto #Catania, ultimat...   

                                               clean_text  score  
178197      varoufakis lâeur cas fiamm buon ragion uscirn      1  
260676  african senz bigliett blocc tren ora httpstcoq...      0  
65993            scoss reatin magnitud httpstcorz4qmmb1h0      0  
252478  nessun tuon nessun fulmin semplic scrosc abbon...      1  
120612  ultim recuper campan chies madonn carmel santâ...      1  


ValueError: Could not load model MilaNLProc/feel-it-italian-sentiment with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.camembert.modeling_camembert.CamembertForSequenceClassification'>). See the original errors:

while loading with AutoModelForSequenceClassification, an error is thrown:
Traceback (most recent call last):
  File "C:\Users\Mega-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\base.py", line 292, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Mega-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\auto\auto_factory.py", line 571, in from_pretrained
    return model_cla

In [58]:

from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# TF-IDF vectorisation
vectorizer = TfidfVectorizer(max_features=1000)
X_vect = vectorizer.fit_transform(df['clean_text'])

# Labels binaires
y = df['score']  # 0 ou 1

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.2, random_state=42)

# Entraînement du modèle SVM
model = SVC(kernel='linear')  
model.fit(X_train, y_train)

# Prédiction et évaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.87      0.94      0.90       735
           1       0.79      0.60      0.68       265

    accuracy                           0.85      1000
   macro avg       0.83      0.77      0.79      1000
weighted avg       0.84      0.85      0.84      1000

